In [ ]:
import time
import MetaTrader5 as mt5
from tensorflow.keras.models import load_model
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import os
import requests

# Load the trained model
model = load_model("lstm_model.keras")

# Define your interval in minutes
interval = 15

# Telegram bot settings
base_url = "https://api.telegram.org/bot7318185424:AAHfIWCYPdDNWx-e_m0qxElxEdhS4opZM1s/sendMessage"
chat_id = "-1002171247293"

# Preprocess data function
def preprocess_data(data, scaler):
    # Converting from object to datetime
    data['time'] = pd.to_datetime(data['time'])

    # Calculating log return
    data['log_return'] = np.log(1 + data['close'].pct_change())

    # Dropping missing values
    data.dropna(inplace=True)

    # Selecting relevant features
    X = data[['close', 'log_return', 'open']]

    # Scaling the features
    X_scaled = scaler.transform(X)

    # Preparing the data for LSTM
    timestep = 5
    X_new = []

    for step in range(timestep, len(X_scaled)):
        X_new.append(X_scaled[step - timestep: step, : X_scaled.shape[1]])

    X_new = np.array(X_new)

    if X_new.size == 0:  # Handle empty X_new
        print("X_new is empty after reshaping. Skipping prediction.")
        return None

    X_new = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], X_new.shape[2]))

    return X_new

# Collect latest data function
def collect_latest_data(symbol, timeframe, num_bars=1):
    # Initialize MT5
    if not mt5.initialize():
        print("initialize() failed")
        return None

    # Check if the symbol is available
    if not mt5.symbol_select(symbol, True):
        print(f"Failed to select symbol: {symbol}")
        mt5.shutdown()
        return None

    # Get historical data
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None or len(rates) == 0:
        print(f"Failed to get rates for {symbol}. Error code: {mt5.last_error()}")
        mt5.shutdown()
        return None

    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    
    # Shutdown MT5
    mt5.shutdown()
    return data

# Send update to Telegram
def send_telegram_update(actual_price, predicted_price):
    message = f"Actual price: {actual_price:.2f}\nPredicted close price: {predicted_price:.2f}"
    parameters = {
        'chat_id': chat_id,
        'text': message
    }
    resp = requests.get(base_url, params=parameters)
    print(f"Status Code: {resp.status_code}")
    print(f"Response Text: {resp.text}")

# Prediction loop
def predict_and_trade():
    scaler = MinMaxScaler(feature_range=(0, 1))

    while True:
        now = datetime.now()
        if now.minute % interval == 0 and now.second == 0:
            # Collect the latest data
            new_data = collect_latest_data("BTCUSD", mt5.TIMEFRAME_M15, 100)  

            if new_data is not None and not new_data.empty:
                # Calculate log return
                new_data['log_return'] = np.log(1 + new_data['close'].pct_change())
                new_data.dropna(inplace=True)
                
                # Fit the scaler on the new data including log return
                scaler.fit(new_data[['close', 'log_return', 'open']])

                # Preprocess the data
                X_new = preprocess_data(new_data, scaler)

                if X_new is not None and X_new.shape[0] > 0:
                    # Make a prediction
                    prediction = model.predict(X_new)

                    # Inverse scale the prediction
                    prediction_scaled = np.c_[prediction, np.zeros(prediction.shape), np.zeros(prediction.shape)]
                    prediction_price = scaler.inverse_transform(prediction_scaled)[:, 0]

                    # Get the actual price
                    actual_price = new_data['close'].iloc[-1]

                    # Send update to Telegram
                    send_telegram_update(actual_price, prediction_price[-1])

            # Sleep to avoid multiple executions within the same interval
            time.sleep(60)
        time.sleep(1)

# Start the prediction loop
predict_and_trade()


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/stepWARNING:tensorflow:5 out of the last 54 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000174C3B26A20> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 958ms/step
Status Code: 200
Response Text: {"ok":true,"result":{"message_id":4,"sender_chat":{"id":-1002171247293,"title":"ShoaibHoque_BTCUSD_Predictions","type":"channel"},"chat":{"id":-1002171247293,"title"

KeyboardInterrupt: 

In [1]:
import time
import MetaTrader5 as mt5
from tensorflow.keras.models import load_model
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import os
import requests

# Load the trained model
model = load_model("lstm_model.keras")

# Define your interval in minutes
interval = 15

# Telegram bot settings
base_url = "https://api.telegram.org/bot7318185424:AAHfIWCYPdDNWx-e_m0qxElxEdhS4opZM1s/sendMessage"
chat_id = "-1002171247293"

# Preprocess data function
def preprocess_data(data, scaler):
    # Converting from object to datetime
    data['time'] = pd.to_datetime(data['time'])

    # Calculating log return
    data['log_return'] = np.log(1 + data['close'].pct_change())

    # Dropping missing values
    data.dropna(inplace=True)

    # Selecting relevant features
    X = data[['close', 'log_return', 'open']]

    # Scaling the features
    X_scaled = scaler.transform(X)

    # Preparing the data for LSTM
    timestep = 5
    X_new = []

    for step in range(timestep, len(X_scaled)):
        X_new.append(X_scaled[step - timestep: step, : X_scaled.shape[1]])

    X_new = np.array(X_new)

    if X_new.size == 0:  # Handle empty X_new
        print("X_new is empty after reshaping. Skipping prediction.")
        return None

    X_new = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], X_new.shape[2]))

    return X_new

# Collect latest data function
def collect_latest_data(symbol, timeframe, num_bars=1):
    # Initialize MT5
    if not mt5.initialize():
        print("initialize() failed")
        return None

    # Check if the symbol is available
    if not mt5.symbol_select(symbol, True):
        print(f"Failed to select symbol: {symbol}")
        mt5.shutdown()
        return None

    # Get historical data
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None or len(rates) == 0:
        print(f"Failed to get rates for {symbol}. Error code: {mt5.last_error()}")
        mt5.shutdown()
        return None

    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    
    return data

# Send update to Telegram
def send_telegram_update(actual_price, predicted_price):
    message = f"Actual price: {actual_price:.2f}\nPredicted close price: {predicted_price:.2f}"
    parameters = {
        'chat_id': chat_id,
        'text': message
    }
    resp = requests.get(base_url, params=parameters)
    return resp

# Execute trade function
def execute_trade(predicted_price):
    # Check if MT5 is initialized and reinitialize if necessary
    if not mt5.initialize():
        print("Failed to initialize MT5.")
        return None

    # Ensure the symbol is selected
    if not mt5.symbol_select("BTCUSD", True):
        print("Failed to select symbol BTCUSD.")
        mt5.shutdown()
        return None

    symbol_info_tick = mt5.symbol_info_tick("BTCUSD")
    if symbol_info_tick is None:
        print("Failed to get symbol info tick for BTCUSD.")
        mt5.shutdown()
        return None

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": "BTCUSD",
        "volume": 0.1,
        "type": mt5.ORDER_TYPE_BUY,
        "price": symbol_info_tick.ask,
        "tp": predicted_price,
        "deviation": 20,
        "magic": 234000,
        "comment": "BTCUSD Prediction",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    mt5.shutdown()
    return result

# Prediction loop
def predict_and_trade():
    scaler = MinMaxScaler(feature_range=(0, 1))

    while True:
        now = datetime.now()
        if now.minute % interval == 0 and now.second == 0:
            # Collect the latest data
            new_data = collect_latest_data("BTCUSD", mt5.TIMEFRAME_M15, 100)  # Adjust the number of bars as needed for preprocessing

            if new_data is not None and not new_data.empty:
                # Calculate log return
                new_data['log_return'] = np.log(1 + new_data['close'].pct_change())
                new_data.dropna(inplace=True)
                
                # Fit the scaler on the new data including log return
                scaler.fit(new_data[['close', 'log_return', 'open']])

                # Preprocess the data
                X_new = preprocess_data(new_data, scaler)

                if X_new is not None and X_new.shape[0] > 0:
                    # Make a prediction
                    prediction = model.predict(X_new)

                    # Inverse scale the prediction
                    prediction_scaled = np.c_[prediction, np.zeros(prediction.shape), np.zeros(prediction.shape)]
                    prediction_price = scaler.inverse_transform(prediction_scaled)[:, 0]

                    # Get the actual price
                    actual_price = new_data['close'].iloc[-1]

                    # Send update to Telegram
                    send_telegram_update(actual_price, prediction_price[-1])

                    # Execute the trade
                    trade_result = execute_trade(prediction_price[-1])
                    print(f"Trade Result: {trade_result}")

            # Sleep to avoid multiple executions within the same interval
            time.sleep(60)
        time.sleep(1)

# Start the prediction loop
predict_and_trade()


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 645ms/step
Trade Result: OrderSendResult(retcode=10009, deal=520939354, order=684931917, volume=0.1, price=61096.32, bid=61081.32, ask=61096.32, comment='Request executed', request_id=4204310901, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='BTCUSD', volume=0.1, price=61096.32, stoplimit=0.0, sl=0.0, tp=61147.73410813809, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='BTCUSD Prediction', position=0, position_by=0))
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step 
Trade Result: OrderSendResult(retcode=10016, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid stops', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='BTCUSD', volume=0.1, price=61200.35, stoplimit=0.0, sl=0.0, tp=61153.16746351481, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='BTCUSD Prediction', position=0, position_by=0))
3/3 ━━━━━━━━━━━━━━━━━━━